In [1]:
# importing necessary packages
import numpy as np
from scipy.io import loadmat
from sklearn.metrics import accuracy_score

In [2]:
#initialise constants

# error-rate is set
error = 20

#learning rate
alpha = 0.001

# add machine epsilon while finding log to avoid log(0)
eps = np.finfo(float).eps

# size of subset of training data to used. If it is none then whole of training set is used
subsetSize = None

# the threshold probability
p0 = 0.7

# set the number of epochs for gradient descent
epoch = 1

In [3]:
#loading and preprocessing of data
X_train = loadmat('mnistTrainImages.mat')
y_train = loadmat('mnistTrainLabels.mat')
    
if subsetSize is not None and subsetSize < X_train['trainData'].shape[0]:
    trainIndices = np.random.randint(0,X_train['trainData'].shape[0],subsetSize)
else:
    trainIndices = range(X_train['trainData'].shape[0])

#convert to numpy array
X_train = np.array(X_train['trainData'][trainIndices,:])
y_train = np.array(y_train['trainLabels'][trainIndices,:])

# prepend with a column of ones to account for bias and take transpose 
X_train = np.transpose(np.insert(X_train,0,1,1))

trainSize = X_train.shape[1]
featurevecSize = X_train.shape[0]
print featurevecSize

785


In [4]:
def calculateLogits(data,theta):
    return 1/(1+np.exp(-1*np.matmul(np.transpose(data),theta)))

In [6]:
def gradientDescent(alpha,error):
    # initialize theta to a random value
    thetaNew = np.random.rand(featurevecSize,1)
    thetaOld = np.inf
    numIterations = 0
    #print np.linalg.norm(thetaNew - thetaOld,1),error
    while np.linalg.norm(thetaNew - thetaOld,2) > error or np.isnan(np.linalg.norm(thetaNew - thetaOld,2)):
        #print np.linalg.norm(thetaNew - thetaOld,2)
        thetaOld = thetaNew
        thetaNew = thetaOld - (alpha * np.matmul(X_train,(calculateLogits(X_train,thetaOld)-y_train)))
        numIterations+=1
    return (thetaNew,numIterations)

In [7]:
def calculateLogitsIndividual(xi,theta):
    return 1/(1+np.exp(-1*np.matmul(np.transpose(theta),xi)))

In [19]:
def stochasticGradientDescent(alpha,epochs):
    # initialize theta to a random value
    thetaNew = np.random.rand(featurevecSize,1)
    thetaOld = np.inf
    #print np.linalg.norm(thetaNew - thetaOld,1),error
    
    #print trainData.shape,trainLabel.shape
    
    for ep in range(epochs):
        data = np.hstack((np.transpose(X_train),y_train))
        np.random.shuffle(data)
        trainData = np.transpose(data[:,0:X_train.shape[1]])
        trainLabel = data[:,X_train.shape[1]:X_train.shape[1]+1]
        for i in range(trainSize) :
            thetaNew = thetaNew - (alpha * np.matmul(X_train[:,i],X_train[:,i]*(calculateLogitsIndividual(X_train[:,i],thetaNew)-y_train[i])))
    return thetaNew

In [9]:
#load test data and test labels
X_test = loadmat('mnistTestImages.mat')
y_test = loadmat('mnistTestLabels.mat')

X_test = np.array(X_test['testData'])
y_test = np.array(y_test['testLabels'])

X_test = np.insert(X_test,0,1,1)

In [44]:
# train the model using gradient descent and then test for accuracy

(theta,numIterations) = gradientDescent(alpha,error)

prediction = calculateLogits(np.transpose(X_test),theta)

positiveClass = np.where(prediction>=p0)[0]
negativeClass = np.where(prediction<p0)[0]

accuracy = float(len(np.where(y_test[positiveClass] == 1)[0]) + len(np.where(y_test[negativeClass] != 1)[0]))*100/y_test.shape[0]

print 'For error tolerance of',error,'with learning rate of',alpha,'gradient descent converged in',numIterations,'iterations'
print 'Accuracy in percentage',accuracy

/home/varshaneya/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


For error tolerance of 20 with learning rate of 0.001 gradient descent converged in 7 iterations
Accuracy in percentage 97.21


### Observations:

Alpha stands for learning rate and error tolerance is the distance (as measured by 2-norm) between successive theta values. These are called hyper-parameters because they are not estimated in the process of regression, but are fixed by the user before the start of the algorithm. As of now there is no closed form formula for calculating them but are set by trial and error. 

The following are accuracies and iterations to converge for various values of hyper-parameters alpha and error tolerance for threshold probability 0.5:

|Error tolerance | Learning rate | Accuracy (%) | Iterations to Converge |
|:---------:|:-------------:|:------------:|:----------------------:|
| 15 | 0.1 | 99.05 | 1096 |
| 20 | 0.1 | 99.12 | 387 |
| 5 | 0.01 | 99.04 | 132 |
| 10 | 0.01 | 98.75 | 58 |
| 15 | 0.01 | 98.38 | 38 |
| 20 | 0.01 | 98.16 | 27 |
| 5 | 0.001 | 97.32 | 10 |
| 10 | 0.001 | 97.35 | 9 |
| 15 | 0.001 | 97.37 | 8 |
| 20 | 0.001 | 97.21 | 7 |

The following are accuracies and iterations to converge for various values of hyper-parameters alpha and error tolerance for threshold probability 0.2:

|Error tolerance | Learning rate | Accuracy (%) | Iterations to Converge |
|:---------:|:-------------:|:------------:|:----------------------:|
| 15 | 0.1 | 99.06 | 1180 |
| 20 | 0.1 | 99.12 | 387 |
| 5 | 0.01 | 99.04 | 132 |
| 10 | 0.01 | 98.75 | 58 |
| 15 | 0.01 | 98.38 | 38 |
| 20 | 0.01 | 98.16 | 27 |
| 5 | 0.001 | 97.33 | 10 |
| 10 | 0.001 | 97.34 | 9 |
| 15 | 0.001 | 97.35 | 8 |
| 20 | 0.001 | 97.24 | 7 |

The following are accuracies and iterations to converge for various values of hyper-parameters alpha and error tolerance for threshold probability 0.8:

|Error tolerance | Learning rate | Accuracy (%) | Iterations to Converge |
|:---------:|:-------------:|:------------:|:----------------------:|
| 15 | 0.1 | 99.0 | 1084 |
| 20 | 0.1 | 99.12 | 387 |
| 5 | 0.01 | 99.04 | 132 |
| 10 | 0.01 | 98.75 | 58 |
| 15 | 0.01 | 98.38 | 38 |
| 20 | 0.01 | 98.17 | 27 |
| 5 | 0.001 | 97.33 | 10 |
| 10 | 0.001 | 97.32 | 9 |
| 15 | 0.001 | 97.34 | 8 |
| 20 | 0.001 | 97.24 | 7 |

1. It is clearly observed that for a lower learning, rate convergence of logits is quite faster. This is because alpha scales down the gradient value thus preventing a lot of oscillations. So it is ideal to choose a smaller learning rate.

2. There is also a steady dip in accuracy along with decreasing learning rate. Hence there is a trade-off between speed and accuracy. That is for a lower learning rate the rate of convergence is fast but accuracy is low and vice versa. This is a common situation that one encounters in various other branches of computer science as well.

3. With an increase in error tolerance the accuracy decreases slightly but the convergence happens faster and vice versa.

4. From the above table ideal values for the parameters are alpha equals 0.1 and error tolerance equals 20 for this dataset. The hyper-parameters vary across datasets.

5. Comparing across the tables we seen that the accuracies across various hyper-parameters **do not change much**. So we can conclude that the probabilities calculated by the sigmoid function are **well-separated** in the sense that they are well *below 0.2* and *above 0.8* for this dataset.

In [22]:
#train using stochastic gradient descent method
theta = stochasticGradientDescent(alpha,epoch)

prediction = calculateLogits(np.transpose(X_test),theta)

positiveClass = np.where(prediction>=p0)[0]
negativeClass = np.where(prediction<p0)[0]

accuracy = float(len(np.where(y_test[positiveClass] == 1)[0]) + len(np.where(y_test[negativeClass] != 1)[0]))*100/y_test.shape[0]

print 'For a learning rate of',alpha,'with number of epochs to be',epoch,'accuracy in percentage',accuracy

For a learning rate of 0.001 with number of epochs to be 1 accuracy in percentage 90.2
